<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# [Static Trumpet Black Hole](https://arxiv.org/abs/1403.5484) Initial data

## Authors: Terrence Pierre Jacques, Zach Etienne & Ian Ruchlin
### Formatting improvements courtesy Brandon Clark

## This module sets up Static Trumpet Black Hole initial data ([Dennison and Baumgarte, 2014 Class. Quantum Grav. 31 117001](https://arxiv.org/abs/1403.5484)). 

### We can convert from spherical to any coordinate system defined in [reference_metric.py](../edit/reference_metric.py) (e.g., SinhSpherical, Cylindrical, Cartesian, etc.) using the [Exact ADM Spherical-or-Cartesian-to-BSSNCurvilinear converter module](Tutorial-ADM_Initial_Data-Converting_Exact_ADM_Spherical_or_Cartesian_to_BSSNCurvilinear.ipynb)

**Notebook Status:** <font color='green'><b> Validated </b></font>

**Validation Notes:** This module has been validated to exhibit convergence to zero of the Hamiltonian constraint violation at the expected order to the exact solution (see plot at bottom of [the exact initial data validation start-to-finish tutorial notebook](Tutorial-Start_to_Finish-BSSNCurvilinear-Setting_up_Exact_Initial_Data.ipynb); momentum constraint is zero), and all quantities have been validated against the [original SENR code](https://bitbucket.org/zach_etienne/nrpy).

### NRPy+ Source Code for this module: [BSSN/StaticTrumpet.py](../edit/BSSN/StaticTrumpet.py)

## Introduction:
These initial data are derived from a family of analytical coordinate systems representing the Schwarzschild spacetime. The coordinates extend smoothly through the black hole event horizon, the spatial coordinates are isotropic (so that the spatial metric can be written as a conformal factor to some power times a flat spatial metric), and, for almost all members of the family, the spatial slices take a so-called $\textit{trumpet geometry}$. Moreover, all expressions are surprisingly simple. This module sets the static trumpet black hole at the origin.

<a id='toc'></a>

# Table of Contents:  
$$\label{toc}$$

1. [Step 1](#initialize_nrpy): Set up the needed NRPy+ infrastructure and declare core gridfunctions
1. [Step 2](#conformal_factor_psi): The conformal factor $\psi$
    1. [Step 2.a](#define_psi): Define the conformal factor $\psi$
    1. [Step 2.b](#nonzero_gamma): Define and construct nonzero components of $\gamma_{ij}$
1. [Step 3](#extrinsic_curvature): Define and construct nonzero components of the extrinsic curvature $K_{ij}$, at the radius $R_0 = M$
1. [Step 4](#lapse_shift): Construct Lapse function $\alpha$ and components of shift vector $\beta$
1. [Step 5](#code_validation): Code Validation against `BSSN.StaticTrumpet` NRPy+ module
1. [Step 6](#latex_pdf_output): Output this notebook to $\LaTeX$-formatted PDF file

<a id='initialize_nrpy'></a>

# Step 1: Set up the needed NRPy+ infrastructure and declare core gridfunctions \[Back to [top](#toc)\]
$$\label{initialize_nrpy}$$

First, we will import the core modules of Python/NRPy+ and specify the main gridfunctions that we will need. 

**Input for initial data**:

* The black hole mass $M$.


In [1]:
# Step P0: Load needed modules
import sympy as sp             # SymPy: The Python computer algebra package upon which NRPy+ depends
import NRPy_param_funcs as par # NRPy+: Parameter interface
import indexedexp as ixp       # NRPy+: Symbolic indexed expression (e.g., tensors, vectors, etc.) support

# All gridfunctions will be written in terms of spherical coordinates (r, th, ph):
r,th,ph = sp.symbols('r th ph', real=True)

thismodule = "StaticTrumpet"

# Step 0: Set spatial dimension (must be 3 for BSSN)
DIM = 3
par.set_parval_from_str("grid::DIM",DIM)

# Step 1: Set psi, the conformal factor:
# Input parameters:
M = par.Cparameters("REAL", thismodule, ["M"], 1.0)

<a id='conformal_factor_psi'></a>

# Step 2: The conformal factor $\psi$ \[Back to [top](#toc)\]
$$\label{conformal_factor_psi}$$

<a id='define_psi'></a>

## Step 2.a: Define the conformal factor $\psi$ \[Back to [top](#toc)\]
$$\label{define_psi}$$


The conformal factor, defined in equation 13 of [Dennison and Baumgarte (2014)](https://arxiv.org/abs/1403.5484), setting $R_0 = M$,
$$ \psi = \sqrt{1 + \frac{M}{r}}. $$

In [2]:
# Step 1: Set psi, the StaticTrumpet conformal factor
# Dennison and Baumgarte (2014) Eq. 13
# https://arxiv.org/pdf/1403.5484.pdf

# psi = sqrt{1 + M/r }
psi0 = sp.sqrt(1 + M/r)

<a id='nonzero_gamma'></a>

## Step 2.b: Define and construct nonzero components of $\gamma_{ij}$ \[Back to [top](#toc)\]
$$\label{nonzero_gamma}$$

The spatial metric, defined in equation 15 of [Dennison and Baumgarte (2014)](https://arxiv.org/abs/1403.5484),
$$ \gamma_{ij} = \psi^4 \eta_{ij}, $$

where $\eta_{ij}$ is the flat metric in spherical polar coordinates.

In [3]:
# *** The physical spatial metric in spherical basis ***
# Set the upper-triangle of the matrix...
# Eq. 15
# eta_00 = 1, eta_11 = r^2, eta_22 = r^2 * sin^2 (theta)
gammaSphDD = ixp.zerorank2()
gammaSphDD[0][0] = psi0**4
gammaSphDD[1][1] = psi0**4 * r**2
gammaSphDD[2][2] = psi0**4 * r**2*sp.sin(th)**2

<a id='extrinsic_curvature'></a>

# Step 3: Define and construct nonzero components of the extrinsic curvature $K_{ij}$, at the radius $R_0 = M$ \[Back to [top](#toc)\]
$$\label{extrinsic_curvature}$$

Components of the extrinsic curvature in spherical basis, defined in equations 19 and 20 of [Dennison and Baumgarte (2014)](https://arxiv.org/abs/1403.5484),

$$ K_{rr} = - \frac{r \left( M-R_0 \right) + MR_0}{r^2 f_1}, $$

<br>

$$ K_{\theta\theta} = \frac{K_{\phi\phi}}{\sin^2 \theta} = f_1, $$

<br>

where $f_1 = \sqrt{2r \left( M-R_0 \right) + R_0 \left( 2M-R_0 \right).}$ Setting $R_0 = M$, these equations reduce to

<br>

$$ K_{rr} = -\frac{M}{r^2}, $$

<br>

$$ K_{\theta\theta} = \frac{K_{\phi\phi}}{\sin^2 \theta} = M. $$


In [4]:
# *** The physical trace-free extrinsic curvature in spherical basis ***
# Set the upper-triangle of the matrix...

# Eq.19 and 20
KSphDD = ixp.zerorank2()

# K_{rr} = - M / r^2
KSphDD[0][0] = -M / r**2

# K_{theta theta} = K_{phi phi} / sin^2 theta = M
KSphDD[1][1] = M

KSphDD[2][2] = M * sp.sin(th)**2

<a id='lapse_shift'></a>

# Step 4: Construct Lapse function $\alpha$ and components of shift vector $\beta$\[Back to [top](#toc)\]
$$\label{lapse_shift}$$

Laspe function and shift vector components, equation 15 of [Dennison and Baumgarte (2014)](https://arxiv.org/abs/1403.5484), setting $R_0 = M$,
$$ \alpha = \frac{r}{r+M}, $$
<br>
$$ \beta^r = \frac{Mr}{\left (r+M \right)^2}, $$
<br>
$$\beta^\theta = \beta^\phi = 0. $$

In [5]:
# Eq. 15
# alpha = r / (r+M)
alphaSph = r / (r + M)

betaSphU = ixp.zerorank1()
# beta^r = Mr / (r + M)^2
betaSphU[0] = M*r / (r + M)**2

BSphU = ixp.zerorank1()

<a id='code_validation'></a>

# Step 5: Code Validation against `BSSN.StaticTrumpet` NRPy+ module \[Back to [top](#toc)\]
$$\label{code_validation}$$

Here, as a code validation check, we verify agreement in the SymPy expressions for static trumpet black hole initial data between

1. this tutorial and 
2. the NRPy+ [BSSN.StaticTrumpet](../edit/BSSN/StaticTrumpet.py) module.

In [6]:
# First we import reference_metric, which is
#   needed since BSSN.BrillLindquist calls
#   BSSN.ADM_Exact_Spherical_or_Cartesian_to_BSSNCurvilinear, which
#   depends on reference_metric:
import reference_metric as rfm   # NRPy+: Reference metric support
rfm.reference_metric()

import BSSN.StaticTrumpet as st
st.StaticTrumpet()

print("Consistency check between Brill-Lindquist tutorial and NRPy+ BSSN.BrillLindquist module. ALL SHOULD BE ZERO.")
print("alphaSph - st.alphaSph = "+str(sp.simplify(alphaSph - st.alphaSph)))
for i in range(DIM):
    print("betaSphU["+str(i)+"] - st.betaSphU["+str(i)+"] = "+\
          str(sp.simplify(betaSphU[i] - st.betaSphU[i])))
    print("BSphU["+str(i)+"] - st.BaSphU["+str(i)+"] = "+str(sp.simplify(BSphU[i] - st.BSphU[i])))
    for j in range(DIM):
        print("gammaSphDD["+str(i)+"]["+str(j)+"] - st.gammaSphDD["+str(i)+"]["+str(j)+"] = "+\
              str(sp.simplify(gammaSphDD[i][j] - st.gammaSphDD[i][j])))
        print("KSphDD["+str(i)+"]["+str(j)+"] - st.KSphDD["+str(i)+"]["+str(j)+"] = "+\
              str(sp.simplify(KSphDD[i][j] - st.KSphDD[i][j])))

Consistency check between Brill-Lindquist tutorial and NRPy+ BSSN.BrillLindquist module. ALL SHOULD BE ZERO.
alphaSph - st.alphaSph = 0
betaSphU[0] - st.betaSphU[0] = 0
BSphU[0] - st.BaSphU[0] = 0
gammaSphDD[0][0] - st.gammaSphDD[0][0] = 0
KSphDD[0][0] - st.KSphDD[0][0] = 0
gammaSphDD[0][1] - st.gammaSphDD[0][1] = 0
KSphDD[0][1] - st.KSphDD[0][1] = 0
gammaSphDD[0][2] - st.gammaSphDD[0][2] = 0
KSphDD[0][2] - st.KSphDD[0][2] = 0
betaSphU[1] - st.betaSphU[1] = 0
BSphU[1] - st.BaSphU[1] = 0
gammaSphDD[1][0] - st.gammaSphDD[1][0] = 0
KSphDD[1][0] - st.KSphDD[1][0] = 0
gammaSphDD[1][1] - st.gammaSphDD[1][1] = 0
KSphDD[1][1] - st.KSphDD[1][1] = 0
gammaSphDD[1][2] - st.gammaSphDD[1][2] = 0
KSphDD[1][2] - st.KSphDD[1][2] = 0
betaSphU[2] - st.betaSphU[2] = 0
BSphU[2] - st.BaSphU[2] = 0
gammaSphDD[2][0] - st.gammaSphDD[2][0] = 0
KSphDD[2][0] - st.KSphDD[2][0] = 0
gammaSphDD[2][1] - st.gammaSphDD[2][1] = 0
KSphDD[2][1] - st.KSphDD[2][1] = 0
gammaSphDD[2][2] - st.gammaSphDD[2][2] = 0
KSphDD[2][2] -

<a id='latex_pdf_output'></a>

# Step 6: Output this notebook to $\LaTeX$-formatted PDF \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

The following code cell converts this Jupyter notebook into a proper, clickable $\LaTeX$-formatted PDF file. After the cell is successfully run, the generated PDF may be found in the root NRPy+ tutorial directory, with filename
[Tutorial-ADM_Initial_Data-StaticTrumpet.pdf](Tutorial-ADM_Initial_Data-StaticTrumpet.pdf) (Note that clicking on this link may not work; you may need to open the PDF file through another means.)

In [7]:
import cmdline_helper as cmd    # NRPy+: Multi-platform Python command-line interface
cmd.output_Jupyter_notebook_to_LaTeXed_PDF("Tutorial-ADM_Initial_Data-StaticTrumpet")

Created Tutorial-ADM_Initial_Data-StaticTrumpet.tex, and compiled LaTeX
    file to PDF file Tutorial-ADM_Initial_Data-StaticTrumpet.pdf
